<a href="https://colab.research.google.com/github/subaruosawa/network-analyze/blob/main/draw_network(part).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json
import datetime
import networkx as nx
import matplotlib.pyplot as plt
from community import community_louvain
import community

In [ ]:
with open("/content/drive/My Drive/jaccard.json") as f:
  jaccard = json.load(f)
with open("/content/drive/My Drive/flow.json") as f:
  flow = json.load(f)
with open("/content/drive/My Drive/aId_3rd_set.json") as f:
  aId = json.load(f)
with open("/content/drive/My Drive/liver_sex.json") as f:
  sex = json.load(f)
with open("/content/drive/My Drive/liver_debut.json") as f:
  debut_str = json.load(f)
with open("/content/drive/My Drive/real_name.json") as f:
  real_name = json.load(f)

In [ ]:
debut = []
for i in debut_str:
  debut.append(datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).date()
   - datetime.datetime.strptime(i,'%Y-%m-%d').date())

In [ ]:
with open("/content/drive/My Drive/real_name.json") as f:
  name_alpha = json.load(f)
with open("/content/drive/My Drive/real_name.json") as f:
  name_beta = json.load(f)

In [ ]:
G = nx.Graph()

for i in range(106):
  G.add_node(real_name[i],
             count = len(aId[i]),
             sex = sex[i],
             debut = debut[i],
             name = real_name[i])

n = 0
del name_alpha[-1]
del name_beta[0]
for alpha in name_alpha:
  for beta in name_beta:
    if not jaccard[2][n] = 0 
      G.add_edge(alpha,beta,weight = jaccard[2][n])
    n += 1
  del name_beta[0]

In [ ]:
#オプション：デビュー時期
#for n,d in list(G.nodes(data=True)):
  #if d["sex"] == "man":
    #G.remove_node(n)

#オプション：性別
#for n,d in list(G.nodes(data=True)):
  #if d["debut"] >= datetime.timedelta(365):
    #G.remove_node(n)

for u,v,d in list(G.edges(data=True)):
  if d["weight"] <= 0.02:
    G.remove_edge(u,v)

for name,deg in list(G.degree):
  if deg <= 1:
    G.remove_node(name)

In [ ]:
plt.figure(figsize=(15,15))
pos = nx.spring_layout(G, k=0.2)
deg = nx.degree_centrality(G).values()
partition=community.best_partition(G)
nx.draw_networkx(G, pos, with_labels=True)
cliques = nx.find_cliques(G)

node_size = [ d["count"]**1.1 for (n,d) in G.nodes(data=True)]
nx.draw_networkx_nodes(G, pos, node_size=node_size,
                       #node_color=list(deg),
                       node_color=list(partition.values()),
                       cmap=plt.cm.RdYlBu
                       )  
edge_width = [ d['weight']*100 for (u,v,d) in G.edges(data=True)]
nx.draw_networkx_edges(G, pos, edge_color=edge_width, width=edge_width, edge_cmap=plt.cm.cool)

plt.axis('off')
plt.show()

レコメンドシステムの設計

In [ ]:
name_adj = [[]*n for n in range(106)]
name_wei = [[]*n for n in range(106)]
for e,na in enumerate(real_name):
  for k,v in G.adj[na].items():
    name_adj[e].append(k)
    name_wei[e].append(v["weight"])

adj_wei = [{}*n for n in range(106)]
for n in range(106):
  adj_wei[n].update(zip(name_adj[n],name_wei[n]))

input = str(LIVER_NAME)
print("選択ライバーと一番ファン層が類似しているライバー")
order_name = real_name.index(input)
print("ライバー名:"+max(adj_wei[order_name], key=adj_wei[order_name].get))
print("Jaccard係数(被っているファンの多さ):"+max(adj_wei[order_name].values()))

In [ ]:
name = []
degree = []
for k,v in nx.degree_centrality(G).items():
  name.append(k)
  degree.append(v)

import pandas as pd
df = pd.DataFrame({"degree":degree},index=name)
df_sort = df.sort_values(by="degree", ascending=False)

input = str(LIVER_NAME)
print("繋がりがある中で一番他ライバーとも繋がりを持つライバー")
order_name = name.index(input)
input_deg = []
name_adj[[]*n for n in range(106)]
for i in name_adj[order_name]:
  input_deg.append(df_sort.loc[i,"degree"])
adj_deg = dict(zip(name_adj[order_name],input_deg))
print("ライバー名:"+max(adj_deg, key=adj_deg.get))
print("次数(繋がりの数)の大きさ:"+max(adj_deg.values()))

クラスターの分析

In [ ]:
print("クラスター数:"+max(partition.values()))
print("各ライバーの所属クラスター:"+partition)

In [ ]:
#ファン層の被りを調べる
cluster_n = []
for k,v in partition.items():
  if v == 0:#クラスターの数字
    cluster_n.append(k)

for u,v in list(G.edges):
  if not u in cluster_n:
    G.remove_edge(u,v)
  elif not v in cluster_n:
    G.remove_edge(u,v)

for name,deg in list(G.degree):
  if deg <= 0:
    G.remove_node(name)

###############################################################################
plt.figure(figsize=(15,15))
pos = nx.spring_layout(G, k=0.2)
deg = nx.degree_centrality(G).values()
nx.draw_networkx(G, pos, with_labels=True)
cliques = nx.find_cliques(G)

node_size = [ d["count"]**1.1 for (n,d) in G.nodes(data=True)]
nx.draw_networkx_nodes(G, pos, node_size=node_size,node_color=list(deg),cmap=plt.cm.cool)
edge_width = [ d['weight']*100 for (u,v,d) in G.edges(data=True)]
nx.draw_networkx_edges(G, pos,edge_color=edge_width, width=edge_width,edge_cmap=plt.cm.cool)

plt.axis('off')
plt.show()

print("クラスター係数(コミュニティ内がどれほど繋がっているか):"+nx.average_clustering(G))

In [ ]:
#ファンの流出入を調べる
DG = nx.DiGraph()

for i in range(106):
  DG.add_node(name_alpha[i],
             count = len(aId[i]),
             sex = sex[i],
             debut = debut[i],
             name = name_str[i])
n = 0
del name_alpha[-1]
del name_beta[0]
for alpha in name_alpha:
  for beta in name_beta:
    if flow[2][n] > 0:
      DG.add_edge(alpha,beta,weight = flow[2][n])
    elif flow[2][n] < 0:
      DG.add_edge(beta,alpha,weight = -flow[2][n])            
    n += 1
  del name_beta[0]

cluster_n = []
for k,v in partition.items():
  if v == 0:#クラスターの数字
    cluster_n.append(k)
for u,v in list(DG.edges):
  if not u in cluster_n:
    DG.remove_edge(u,v)
  elif not v in cluster_n:
    DG.remove_edge(u,v)

for u,v,d in list(DG.edges(data=True)):
  if d["weight"] <= 100:
    DG.remove_edge(u,v)

for name,deg in list(DG.degree):
  if deg <= 0:
    DG.remove_node(name)
################################################################################
plt.figure(figsize=(15,15))
pos = nx.spring_layout(DG, k=0.2)
deg = nx.degree_centrality(DG).values()
nx.draw_networkx(DG, pos, with_labels=True)

node_size = [ d["count"]/10 for (n,d) in DG.nodes(data=True)]
nx.draw_networkx_nodes(DG, pos, node_size=node_size, node_color=list(deg), cmap=plt.cm.cool)
edge_width = [ d['weight']/30 for (u,v,d) in DG.edges(data=True)]
nx.draw_networkx_edges(DG, pos,edge_color=edge_width, width=edge_width, edge_cmap=plt.cm.cool)

plt.axis('off')
plt.show()

各ライバー周辺のコミュニティ確認

In [ ]:
G = nx.Graph()

for i in range(106):
  G.add_node(name_str[i],
             count = len(aId[i]),
             sex = sex[i],
             debut = debut[i],
             name = name_str[i])

n = 0
del name_alpha[-1]
del name_beta[0]
for alpha in name_alpha:
  for beta in name_beta:
    if not jaccard[2][n] = 0 
      G.add_edge(alpha,beta,weight = jaccard[2][n])
    n += 1
  del name_beta[0]

for u,v,d in list(G.edges(data=True)):
  if d["weight"] <= 0.02:
    G.remove_edge(u,v)

input = str(LIVER_NAME)
first_adj = []
for na in G.adj[pick].keys():
  first_adj.append(na)
adj_set = list(set([pick]+first_adj))
for u,v in list(G.edges):
  if not u in adj_set:
    if not v in adj_set:
      G.remove_edge(u,v)

for name,deg in list(G.degree):
  if deg <= 1:
    G.remove_node(name)

plt.figure(figsize=(15,15))
pos = nx.spring_layout(G, k=0.2)
deg = nx.degree_centrality(G).values()
partition=community.best_partition(G)
nx.draw_networkx(G, pos, with_labels=True)
cliques = nx.find_cliques(G)

node_size = [ d["count"]**1.1 for (n,d) in G.nodes(data=True)]
nx.draw_networkx_nodes(G, pos, node_size=node_size,
                       #node_color=list(deg),
                       node_color=list(partition.values()),
                       cmap=plt.cm.RdYlBu
                       )  
edge_width = [ d['weight']*100 for (u,v,d) in G.edges(data=True)]
nx.draw_networkx_edges(G, pos, edge_color=edge_width, width=edge_width, edge_cmap=plt.cm.cool)

plt.axis('off')
plt.show()